In [1]:
import numpy as np 
import pandas as pd
import seaborn as sns

#set options to read all the rows
pd.set_option('display.max_rows', None)

In [2]:
df = pd.read_csv('jd.csv')
df.head()

,Role,Company,Job_Description
0,Data Scientist,Google,Minimum qualifications:\n\nMaster's degree in ...
1,Data Scientist,Meta,Identify appropriate quantitative methods and ...
2,Business Analyst,Amazon,The US 3P Business Organization is seeking an ...
3,Python Developer,Microsoft,The Azure Big Data SCOPE Compiler / Language t...
4,Software Engineer,Apple,Key Qualifications\nStrong experience on a pro...


In [3]:
df_resume = pd.read_csv('resume.csv')
df_resume.head()

,Category,Resume
0,Data Science,Skills * Programming Languages: Python (pandas...
1,Data Science,Education Details \r\nMay 2013 to May 2017 B.E...
2,Data Science,"Areas of Interest Deep Learning, Control Syste..."
3,Data Science,Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...
4,Data Science,"Education Details \r\n MCA YMCAUST, Faridab..."


In [4]:
import texthero as hero

#clean the text file using texthero
df1 = df.copy()
df1['Job_Description'] = hero.clean(df1['Job_Description'])
df1.head()

,Role,Company,Job_Description
0,Data Scientist,Google,minimum qualifications master degree quantitat...
1,Data Scientist,Meta,identify appropriate quantitative methods buil...
2,Business Analyst,Amazon,us 3p business organization seeking exceptiona...
3,Python Developer,Microsoft,azure big data scope compiler language team lo...
4,Software Engineer,Apple,key qualifications strong experience professio...


In [5]:
import texthero as hero

#clean the text file using texthero
df2 = df_resume.copy()
df2['Resume'] = hero.clean(df2['Resume'])
df2.head()

,Category,Resume
0,Data Science,skills programming languages python pandas num...
1,Data Science,education details may may b e uit rgpv data sc...
2,Data Science,areas interest deep learning control system de...
3,Data Science,skills aC/ r aC/ python aC/ sap hana aC/ table...
4,Data Science,education details mca ymcaust faridabad haryan...


In [6]:
from scipy import spatial # for cosine distance
from sent2vec.vectorizer import Vectorizer # uses DistilBERT

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()

In [8]:
sent1 =  df1['Job_Description'][0]
sent2 = df2['Resume'][0]

corpus = [sent1,sent2]

X_train_counts = count_vect.fit_transform(corpus)

pd.DataFrame(X_train_counts.toarray(),columns=count_vect.get_feature_names(),index=['Document 1','Document 2'])

,ac,accelerating,accounting,achievement,across,action,addresses,ads,advanced,advocating,...,word2vec,words,work,worked,world,written,year,years,young,youtube
Document 1,0,0,0,1,0,0,0,1,1,1,...,0,0,4,0,2,1,0,2,0,1
Document 2,1,1,1,0,3,1,1,0,1,0,...,1,2,0,1,0,0,1,0,2,0


In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()

trsfm=vectorizer.fit_transform(corpus)
pd.DataFrame(trsfm.toarray(),columns=vectorizer.get_feature_names(),index=['Document 1','Document 2'])

,ac,accelerating,accounting,achievement,across,action,addresses,ads,advanced,advocating,...,word2vec,words,work,worked,world,written,year,years,young,youtube
Document 1,0.000000,0.000000,0.000000,0.044938,0.000000,0.000000,0.000000,0.044938,0.031974,0.044938,...,0.000000,0.000000,0.179752,0.000000,0.089876,0.044938,0.000000,0.089876,0.000000,0.044938
Document 2,0.030778,0.030778,0.030778,0.000000,0.092335,0.030778,0.030778,0.000000,0.021899,0.000000,...,0.030778,0.061556,0.000000,0.030778,0.000000,0.000000,0.030778,0.000000,0.061556,0.000000


In [10]:
from sklearn.metrics.pairwise import cosine_similarity

cosine = cosine_similarity(trsfm[0:1], trsfm)
cosine[0,1]*100

19.115196563947507

In [11]:
#define Jaccard Similarity function
def jaccard(list1, list2):
    intersection = len(list(set(list1).intersection(list2)))
    union = (len(list1) + len(list2)) - intersection
    return float(intersection) / union

In [12]:
#find Jaccard Similarity between the two stemmed sets 
jaccard(sent1,sent2)*100

0.4235294117647058

In [13]:
df1['Role_Company'] = df1['Role'].astype(str) +"-"+ df1["Company"]
df1.drop(['Role','Company'],axis=1,inplace=True)

In [14]:
columns = df1['Role_Company'].tolist()
columns

['Data Scientist-Google',
 'Data Scientist-Meta',
 'Business Analyst-Amazon',
 'Python Developer-Microsoft',
 'Software Engineer-Apple',
 'DevOps Engineer-Amazon',
 'ETL Developer-Amazon']

In [15]:
df_similarity=pd.DataFrame(columns=["Resume"]+columns)
df_similarity.head()

,Resume,Data Scientist-Google,Data Scientist-Meta,Business Analyst-Amazon,Python Developer-Microsoft,Software Engineer-Apple,DevOps Engineer-Amazon,ETL Developer-Amazon


In [16]:
job =  df1['Job_Description']
resume = df2['Resume']

columns = list(df_similarity)
data = []

resume_job = []

for r in resume:
    resume_job = []
    resume_job.append(r)
    similarity = []
    for j in job:
        corpus = [r,j]
        trsfm = vectorizer.fit_transform(corpus)
        cosine = cosine_similarity(trsfm[0:1], trsfm)
        cosine_prcnt = cosine[0,1]*100
        similarity.append(cosine_prcnt)
    resume_job.extend(similarity)
    zipped = zip(columns, resume_job)
    a_dictionary = dict(zipped)
    print(a_dictionary)
    data.append(a_dictionary)

{'Resume': 'skills programming languages python pandas numpy scipy scikit learn matplotlib sql java javascript jquery machine learning regression svm naa-ve bayes knn random forest decision trees boosting techniques cluster analysis word embedding sentiment analysis natural language processing dimensionality reduction topic modelling lda nmf pca neural nets database visualizations mysql sqlserver cassandra hbase elasticsearch d3 js dc js plotly kibana matplotlib ggplot tableau others regular expression html css angular logstash kafka python flask git docker computer vision open cv understanding deep learning education details data science assurance associate data science assurance associate ernst young llp skill details javascript exprience months jquery exprience months python exprience monthscompany details company ernst young llp description fraud investigations dispute services assurance technology assisted review tar technology assisted review assists accelerating review process r

{'Resume': 'areas interest deep learning control system design programming python electric machinery web development analytics technical activities q hindustan aeronautics limited bangalore weeks guidance mr satish senior engineer hangar mirage fighter aircraft technical skills programming matlab python java labview python webframework django flask ltspice intermediate languages mipower intermediate github gitbash jupyter notebook xampp mysql basics python software packages interpreters anaconda python2 python3 pycharm java ide eclipse operating systems windows ubuntu debian kali linux education details january b tech electrical electronics engineering manipal institute technology january deeksha center january little flower public school august manipal academy higher data science data science electrical enthusiast skill details data analysis exprience less year months excel exprience less year months machine learning exprience less year months mathematics exprience less year months py

{'Resume': 'skills c basics iot python matlab data science machine learning html microsoft word microsoft excel microsoft powerpoint recognition academic secured first place b tech education details august may b tech ghatkesar andhra pradesh aurora scientific technological institute june may secondary education warangal telangana sr junior college data science skill details ms office exprience less year months c exprience less year months machine learning exprience less year months data science exprience less year months matlab exprience less year monthscompany details company description', 'Data Scientist-Google': 6.975812741784361, 'Data Scientist-Meta': 8.307325017636849, 'Business Analyst-Amazon': 6.581933672377752, 'Python Developer-Microsoft': 6.655422928682728, 'Software Engineer-Apple': 2.8267365120098753, 'DevOps Engineer-Amazon': 0.4825138408269829, 'ETL Developer-Amazon': 5.3672489986290115}
{'Resume': 'skills aC/ python aC/ tableau aC/ data visualization aC/ r studio aC/ ma

{'Resume': 'skills c basics iot python matlab data science machine learning html microsoft word microsoft excel microsoft powerpoint recognition academic secured first place b tech education details august may b tech ghatkesar andhra pradesh aurora scientific technological institute june may secondary education warangal telangana sr junior college data science skill details ms office exprience less year months c exprience less year months machine learning exprience less year months data science exprience less year months matlab exprience less year monthscompany details company description', 'Data Scientist-Google': 6.975812741784361, 'Data Scientist-Meta': 8.307325017636849, 'Business Analyst-Amazon': 6.581933672377752, 'Python Developer-Microsoft': 6.655422928682728, 'Software Engineer-Apple': 2.8267365120098753, 'DevOps Engineer-Amazon': 0.4825138408269829, 'ETL Developer-Amazon': 5.3672489986290115}
{'Resume': 'skills aC/ python aC/ tableau aC/ data visualization aC/ r studio aC/ ma

{'Resume': 'education details june may b economics chennai tamil nadu sdnbvc hr skill details company details company anything solution description hr', 'Data Scientist-Google': 1.1765716210712325, 'Data Scientist-Meta': 0.0, 'Business Analyst-Amazon': 0.0, 'Python Developer-Microsoft': 0.9637440001794413, 'Software Engineer-Apple': 1.0761229541127142, 'DevOps Engineer-Amazon': 0.0, 'ETL Developer-Amazon': 0.9348211145785351}
{'Resume': 'education details june may b economics chennai tamil nadu sdnbvc hr skill details company details company anything solution description hr', 'Data Scientist-Google': 1.1765716210712325, 'Data Scientist-Meta': 0.0, 'Business Analyst-Amazon': 0.0, 'Python Developer-Microsoft': 0.9637440001794413, 'Software Engineer-Apple': 1.0761229541127142, 'DevOps Engineer-Amazon': 0.0, 'ETL Developer-Amazon': 0.9348211145785351}
{'Resume': 'education details bba lovely professional university hr skill details communication exprience monthscompany details company desc

...

{'Resume': 'aC/ good logical analytical skills aC/ positive attitude towards solving problems accepting challenges aC/ team player leader aC/ good organizer presonal details dob marital status single education details july june electrical pune maharashtra sinhagad institute technology january diploma msbte kopargaon maharashtra k b p polytechnic january ssc g vidyalaya state board testing engineer electrical engineer skill details company details company intelux electronics pvt ltd description power management system testing department', 'Data Scientist-Google': 2.8931032266306462, 'Data Scientist-Meta': 1.3835461553249997, 'Business Analyst-Amazon': 1.8092243778877668, 'Python Developer-Microsoft': 3.2493306591757825, 'Software Engineer-Apple': 2.6023117247441303, 'DevOps Engineer-Amazon': 3.6739104375277485, 'ETL Developer-Amazon': 3.4218606272102563}
{'Resume': 'computer proficiency aC/ basic ms office powerpoint word outlook excel aC/ language known basics c cpp java aC/ basics net

{'Resume': 'skill set os windows xp database mysql sql server languages core java web technology html css testing manual testing database testing bug tracking reporting end user handling education details january mcs pune maharashtra p v p college pravaranagar january hsc k junior college january ssc k mahavidyalya jr testing engineer jr testing engineer tech innovations ltd skill details testing exprience months windows xp exprience months css exprience months database exprience months database testing exprience monthscompany details company tech innovations ltd description responsibilities aC/ analyzing testing requirements aC/ preparing test plans test scenarios aC/ preparing test cases aC/ executing test cases aC/ tracking bugs aC/ coordinating developers order fix aC/ preparing test summary reports aC/ reporting quality manager aC/ attending meeting projects project name web based time attendance application environment windows vista windows xp windows windows clients mccia sapphi

{'Resume': 'personal skills aC/ quick learner aC/ eagerness learn new things aC/ competitive attitude aC/ good leadership qualities aC/ ability deal people diplomatically personal dossier fathers name dhanraj wagheducation details january january bachelors engineering engineering pune maharashtra pune university january higher secondary certificate nashik maharashtra snd college engineering research center january secondary school certificate yeola maharashtra swami muktanand jr college hsc maharashtra state board ssc maharashtra state bard rajapur uttar pradesh madhyamik vidya mandir rajapur testing quality control engineer testing quality control engineer rakesh transformer industries pvt ltd skill details company details company rakesh transformer industries pvt ltd description responsibilities aC/ conduct routine test type test temperature rise test special test transformers per iec mva kv class preparing test reports aC/ routine tests type tests special tests per iec test manual i

{'Resume': 'computer skills aC/ proficient ms office word basic excel power point strength aC/ hard working loyalty creativity aC/ self motivated responsible initiative aC/ good people management skill positive attitude aC/ knowledge windows internet education details bachelor electrical engineering electrical engineering nashik maharashtra guru gobind singh college engineering research centre diploma electrical engineering nashik maharashtra e polytechnic college testing engineer skill details excel exprience months ms office exprience months word exprience monthscompany details company description department testing responsibilities aC/ check acb vcb circuit breaker aC/ following test conducted circuit breaker per drawing check breaker timing check contact resistance using contact resistance meter crm check breaker insulation resistance ir check breaker rack rack properly check closing tripping operation work properly aC/ check following test conducted mcc pcc panel insulation resist

{'Resume': 'computer proficiency aC/ basic ms office powerpoint word outlook excel aC/ language known basics c cpp java aC/ basics networking aC/ basics command linux project details minor project details aC/ title applocker android aC/ project area android application aC/ description applocker provides protection system applications well third party applications installed android devices password protection provided help patterns volume keys hence extra measure privacy acquired major project details aC/ title online complaint system cyber crimes aC/ project area android application aC/ description online complaint system cyber crimes android application use person lodged complaint police station regarding cyber crime action taken within prescribed time constraint person directly use application help directly lodge complaint commissioner office get store commissioner database necessary action taken strengths aC/ belief team work team member leader aC/ hard ethical worker education deta

In [17]:
df_similarity = df_similarity.append(data, True)

In [18]:
df_similarity.head()

,Resume,Data Scientist-Google,Data Scientist-Meta,Business Analyst-Amazon,Python Developer-Microsoft,Software Engineer-Apple,DevOps Engineer-Amazon,ETL Developer-Amazon
0,skills programming languages python pandas num...,19.115197,14.651894,12.564862,14.832144,7.887163,5.032347,12.953251
1,education details may may b e uit rgpv data sc...,4.206203,5.376500,4.992131,3.710819,1.329258,1.143470,3.703753
2,areas interest deep learning control system de...,8.016517,8.978215,7.097594,10.097543,5.893798,3.391123,7.409766
3,skills aC/ r aC/ python aC/ sap hana aC/ table...,12.688287,11.118550,14.245513,11.615033,8.014396,5.644845,9.641717
4,education details mca ymcaust faridabad haryan...,7.009412,4.833435,5.485900,5.089924,2.928431,0.376121,4.476053


In [19]:
df_similarity['Max_Match_Prcnt'] = df_similarity.drop(['Resume'],axis=1).max(axis=1)
df_similarity['Max_Match_Job'] = df_similarity.drop(['Resume'],axis=1).idxmax(axis=1)

In [20]:
df_similarity.head()

,Resume,Data Scientist-Google,Data Scientist-Meta,Business Analyst-Amazon,Python Developer-Microsoft,Software Engineer-Apple,DevOps Engineer-Amazon,ETL Developer-Amazon,Max_Match_Prcnt,Max_Match_Job
0,skills programming languages python pandas num...,19.115197,14.651894,12.564862,14.832144,7.887163,5.032347,12.953251,19.115197,Data Scientist-Google
1,education details may may b e uit rgpv data sc...,4.206203,5.376500,4.992131,3.710819,1.329258,1.143470,3.703753,5.376500,Data Scientist-Meta
2,areas interest deep learning control system de...,8.016517,8.978215,7.097594,10.097543,5.893798,3.391123,7.409766,10.097543,Python Developer-Microsoft
3,skills aC/ r aC/ python aC/ sap hana aC/ table...,12.688287,11.118550,14.245513,11.615033,8.014396,5.644845,9.641717,14.245513,Business Analyst-Amazon
4,education details mca ymcaust faridabad haryan...,7.009412,4.833435,5.485900,5.089924,2.928431,0.376121,4.476053,7.009412,Data Scientist-Google
